# Dependency

In [46]:
import pandas as pd
import numpy as np
from pandas_datareader import data as wb # datareader supports multiple financial database including yahoo and google
import datetime
from dateutil.relativedelta import relativedelta
import GetOldTweets3 as tws
import tweepy

# Data Source
## Start & End Date

In [147]:
date_rang_month = 3
start_date = datetime.date.today() + relativedelta(months=-date_rang_month)
end_date = datetime.date.today()

## TESLA Stock

In [156]:
tsla_df = pd.DataFrame(wb.DataReader("TSLA", 
                                    data_source = "yahoo", 
                                    start = start_date, 
                                    end = end_date))
tsla_df = tsla_df.dropna()
tsla_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,459.119995,428.500000,436.092010,447.750000,118465000.0,447.750000
2020-08-28,463.697998,437.303986,459.023987,442.679993,100406000.0,442.679993
2020-08-31,500.140015,440.109985,444.609985,498.320007,118374400.0,498.320007
2020-09-01,502.489990,470.510010,502.140015,475.049988,90119400.0,475.049988
2020-09-02,479.040009,405.119995,478.989990,447.369995,96176100.0,447.369995


## Dow Jones Index Average

In [157]:
djia_df = pd.DataFrame(wb.DataReader("DJIA", 
                                     data_source = "fred",
                                     start = start_date + relativedelta(days = -1), # due to fred database's specialty, we should set the start date one day before the actual start date.
                                     end = end_date))
djia_df = djia_df.dropna()
djia_df.head(5)

,DJIA
DATE,
2020-08-27,28492.27
2020-08-28,28653.87
2020-08-31,28430.05
2020-09-01,28645.66
2020-09-02,29100.50


## Federal Funds Rate  

In [158]:
dff_df = pd.DataFrame(wb.DataReader("DFF", 
                                    data_source = "fred",
                                    start = start_date + relativedelta(days = -1), # due to fred database's specialty, we should set the start date one day before the actual start date.
                                    end = end_date))
dff_df = dff_df.dropna()
dff_df.head(5)

,DFF
DATE,
2020-08-27,0.08
2020-08-28,0.09
2020-08-29,0.09
2020-08-30,0.09
2020-08-31,0.09


## Panasonic Corp. Stock

In [159]:
pcrfy_df = pd.DataFrame(wb.DataReader("PCRFY", 
                                      data_source = "yahoo", 
                                      start = start_date, 
                                      end = end_date))
pcrfy_df = pcrfy_df.dropna()
pcrfy_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,9.21,9.02,9.02,9.09,148200,9.09
2020-08-28,9.30,9.04,9.04,9.27,170600,9.27
2020-08-31,9.33,9.20,9.31,9.29,177800,9.29
2020-09-01,9.29,9.18,9.27,9.23,200400,9.23
2020-09-02,9.22,9.04,9.13,9.22,179500,9.22


## BMW Stock

In [160]:
bwm_df = pd.DataFrame(wb.DataReader("BMW.DE", 
                                    data_source = "yahoo", 
                                    start = start_date, 
                                    end = end_date))
bwm_df = bwm_df.dropna()
bwm_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,60.980000,59.299999,59.500000,59.599998,1836847.0,59.599998
2020-08-28,60.540001,59.349998,60.070000,59.610001,1090761.0,59.610001
2020-08-31,61.770000,59.799999,59.799999,60.150002,2133443.0,60.150002
2020-09-01,61.389999,59.639999,61.259998,60.130001,1566399.0,60.130001
2020-09-02,61.189999,60.110001,60.709999,60.389999,1404861.0,60.389999


## Daimler AG Stock

In [161]:
dai_df = pd.DataFrame(wb.DataReader("DAI.DE", 
                                    data_source = "yahoo", 
                                    start = start_date, 
                                    end = end_date))
dai_df = dai_df.dropna()
dai_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,43.785000,42.930000,43.119999,43.000000,3215726.0,43.000000
2020-08-28,43.465000,42.639999,43.334999,42.880001,2933742.0,42.880001
2020-08-31,43.435001,42.540001,43.000000,42.645000,3328182.0,42.645000
2020-09-01,43.224998,41.770000,43.000000,42.674999,3249098.0,42.674999
2020-09-02,43.369999,42.625000,42.820000,43.080002,3398338.0,43.080002


## XPeng Inc. Stock

In [162]:
xpev_df = pd.DataFrame(wb.DataReader("XPEV", 
                                    data_source = "yahoo", 
                                    start = start_date, 
                                    end = end_date))
xpev_df = xpev_df.dropna()
xpev_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,25.000000,21.200001,23.100000,21.219999,82219700,21.219999
2020-08-28,24.400000,22.020000,23.980000,22.790001,44847300,22.790001
2020-08-31,22.790001,20.330000,22.690001,20.500000,20816000,20.500000
2020-09-01,21.790001,20.100000,20.980000,21.610001,15291400,21.610001
2020-09-02,22.000000,20.379999,21.990000,21.090000,9090100,21.090000


## BYD Company Limited Stock

In [163]:
byddf_df = pd.DataFrame(wb.DataReader("BYDDF", 
                                    data_source = "yahoo", 
                                    start = start_date, 
                                    end = end_date))
byddf_df = byddf_df.dropna()
byddf_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-08-27,10.36,10.00,10.15,10.10,355200,10.10
2020-08-28,10.27,9.96,10.10,10.20,232200,10.20
2020-08-31,10.39,9.99,10.05,10.36,598200,10.36
2020-09-01,11.24,10.93,11.05,11.21,248600,11.21
2020-09-02,11.24,11.10,11.20,11.17,163800,11.17


## Brent Crude Oil Last Day Financ

In [164]:
oil_df = pd.DataFrame(wb.DataReader("BZ=F", 
                                    data_source = "yahoo", 
                                    start = start_date,
                                    end = end_date))
oil_df = oil_df.dropna()
oil_df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-10-02,40.820000,38.790001,40.790001,39.270000,37290.0,39.270000
2020-10-05,41.779999,39.209999,39.220001,41.290001,30456.0,41.290001
2020-10-06,42.840000,41.220001,41.520000,42.650002,37426.0,42.650002
2020-10-07,42.419998,41.360001,41.939999,41.990002,30170.0,41.990002
2020-10-08,43.470001,41.860001,42.060001,43.340000,33351.0,43.340000


## Number of Tesla vehicles delivered worldwide 

In [134]:
quaters = [
    (r'q2', r'2019'),
    (r'q3', r'2019'),
    (r'q4', r'2019'),
    (r'q1', r'2020'),
    (r'q2', r'2020'),
    (r'q3', r'2020')
]

deliver_df = list()
for q, year in quaters:
    url = r'https://ir.tesla.com/press-release/tesla-' + q + r'-' + year + r'-vehicle-production-deliveries'
    tables = pd.read_html(url)
    tmp = list()
    for row in range(1,4):
        for col in range(1, 3):
            if q == r'q2' and year == r'2019':
                tmp.append(tables[0].iloc[row, col + 1])
            else:
                tmp.append(tables[0].iloc[row, col])
    deliver_df.append(tmp)
deliver_df = pd.DataFrame(deliver_df,
                          columns= [
                              "Production Model S/X",
                              "Deliveries Model S/X",
                              "Production Model 3",
                              "Deliveries Model 3",
                              "Production Total",
                              "Deliveries Total",
                         ])
deliver_df
    

,Production Model S/X,Deliveries Model S/X,Production Model 3,Deliveries Model 3,Production Total,Deliveries Total
0,14517,17650,72531,77550,87048,95200
1,16318,17400,79837,79600,96155,97000
2,17933,19450,86958,92550,104891,112000
3,15390,12200,87282,76200,102672,88400
4,6326,10600,75946,80050,82272,90650
5,16992,15200,128044,124100,145036,139300


## Tesla Revenue

In [115]:
revenue_df = pd.read_html(r'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue')
revenue_df = revenue_df[1]
revenue_df.dropna(how = "any")
revenue_df.head(5)

,Tesla Quarterly Revenue(Millions of US $),Tesla Quarterly Revenue(Millions of US $).1
0,2020-09-30,"$8,771"
1,2020-06-30,"$6,036"
2,2020-03-31,"$5,985"
3,2019-12-31,"$7,384"
4,2019-09-30,"$6,303"


# Data Cleaning

Currently, the data used includes:

1. Tesla Stock `tsla_df`
2. Dow Jones Index Average `djia_df`
3. Federal Funds Rate `dff_df`
4. Panasonic Corp. Stock `pcrfy_df`
5. BMW Stock `bwm_df`
6. Daimler AG Stock `dai_df`
7. XPeng Inc. Stock `xpev_df`
8. BYD Company Limited Stock `byddf_df`
9. Brent Crude Oil Last Day Financ `oil_df`
10. Number of Tesla vehicles delivered worldwide `deliver_df`
11. Tesla Revenue `revenue_df`

Actually, in the last section, we have done a bit of data Cleaning, like

1. drop nan/missing values 
2. reset the index of dataframe to include `date` as one column in the dataframe

However, we still need to do more:

1. Renaming columns to a more recognizable set of labels 
2. Dropping unnecessary columns in a DataFrame
3. do some more modification on `deliver_df` and `revenue_df`
4. Combine all these dataframe to a complete DataFrame

## Clean `revenue_df`

In [116]:
# rename the dataframe
revenue_df.columns = ["Date", "Tesla Quarterly Revenue(Millions of US $)"]

# drop rows with missing values
for col in revenue_df.columns:
    if np.sum(revenue_df[col].isna()) != 0:
        print("WARNING: for revenue_df, column: " + col + " appears missing values")
        revenue_df = revenue_df[revenue_df[col].notna()]

# transfer data type of "Date" from string to TimeStamp
revenue_df["Date"] = revenue_df["Date"].map(pd.Timestamp)

# transfer data type of "Tesla Quarterly Revenue(Millions of US $)" from string to decimal
def doller2decimal(target):
    return float(target.replace(",", "").replace("$", ""))
revenue_df["Tesla Quarterly Revenue(Millions of US $)"] = revenue_df["Tesla Quarterly Revenue(Millions of US $)"].apply(doller2decimal)

# reorder the revenue_df to ascending order by Date
revenue_df = revenue_df.sort_values(by = ["Date"], ascending = True)

revenue_df.tail(5)


,Date,Tesla Quarterly Revenue(Millions of US $)
4,2019-09-30,6303.0
3,2019-12-31,7384.0
2,2020-03-31,5985.0
1,2020-06-30,6036.0
0,2020-09-30,8771.0


## Clean `deliver_df`

In [135]:
# add "Date" to the dataframe deliver_df
deliver_df["Date"] = revenue_df["Date"][-len(deliver_df):].reset_index()["Date"]
deliver_df = deliver_df[deliver_df.columns.to_list()[-1:] + deliver_df.columns.to_list()[:-1]] # reorder the columns

deliver_df

,Date,Production Model S/X,Deliveries Model S/X,Production Model 3,Deliveries Model 3,Production Total,Deliveries Total
0,2019-06-30,14517,17650,72531,77550,87048,95200
1,2019-09-30,16318,17400,79837,79600,96155,97000
2,2019-12-31,17933,19450,86958,92550,104891,112000
3,2020-03-31,15390,12200,87282,76200,102672,88400
4,2020-06-30,6326,10600,75946,80050,82272,90650
5,2020-09-30,16992,15200,128044,124100,145036,139300
